### Librerías a Útilizar

---

In [1]:
import numpy as np
import random
import string
import os

### Objeto Tablero

---

**El Objeto Tablero (`Table`) debe de tener las siguientes caracteristicas:**

* Debe generar letras aleatoriamente, desde la A, hasta la Z (Sin la inclusión de la Ñ)
* Se debe de asegurar de que haya posibilidad de combinación de los `keywords`, que son:
    - `LOK`
    - `TLAK`
    - `TAK` 

---

**IDEA**


Intentar localizar las palabras claves `LOK`, `TLAK`, y `TAK` en coordenadas aleatorias, tanto al derecho como alreves en cualquier dirección, y que luego, los demás espacios vacíos se coloquen letras aleatorias.

Esto es para tener mayor seguridad de que el juego sea jugable (valga la redundancia), y que no haya un juego sin alguna de las palabras clave, lo cual sería injugable.

In [2]:
class Table:
    """
    Clase que crea el tablero del juego
    """

    def __init__(self, rows, columns):
        """
        Inicializa el tablero con filas y columnas dadas.
        Inserta palabras clave y rellena con letras aleatorias.
        """
        self.rows = rows
        self.columns = columns
        self.data = np.full((rows, columns), '0', dtype='U1')  # Matriz de letras
        self._valid_letters = list(string.ascii_uppercase)  # Letras válidas (A-Z)
        self.occupied = set()  # Celdas ocupadas por keywords o X
        self.keywords = ['LOK', 'TLAK', 'TA']

        # Pool de palabras clave para insertar
        keywords_pool = ['LOK'] * 3 + ['TLAK'] * 2 + ['TA']

        # Calcular cantidad de palabras clave proporcional al tamaño del tablero
        total_cells = self.rows * self.columns
        num_keywords = max(2, min(total_cells // 6, len(keywords_pool)))  

        inserted_keywords = 0
        attempts = 0 

        # Inserta 3 palabras clave aleatoriamente
        while inserted_keywords < num_keywords and attempts < num_keywords * 10:
            keyword = random.choice(keywords_pool)
            direction = random.choice(['horizontal', 'vertical'])
            reverse = random.choice([True, False])
            if self.insert_keyword(keyword, direction, reversed=reverse):
                inserted_keywords += 1
            attempts += 1

        self.insert_connector_x()  # Inserta conectores X
        self.fill_with_letters()   # Rellena el resto con letras aleatorias

    def insert_keyword(self, word, direction, reversed=False):
        """
        Inserta una palabra clave en el tablero en la dirección dada.
        Retorna True si se pudo insertar, False si no.
        """
        if reversed:
            word = word[::-1]

        max_attempts = 100
        for _ in range(max_attempts):
            if direction == 'horizontal':
                row = random.randint(0, self.rows - 1)
                col = random.randint(0, self.columns - len(word))
                coords = [(row, col + i) for i in range(len(word))]
            elif direction == 'vertical':
                row = random.randint(0, self.rows - len(word))
                col = random.randint(0, self.columns - 1)
                coords = [(row + i, col) for i in range(len(word))]
            else:
                raise ValueError("Invalid direction. Must be 'horizontal' or 'vertical'.")

            # Verifica que las celdas estén libres
            if all((r, c) not in self.occupied for r, c in coords):
                for i, (r, c) in enumerate(coords):
                    self.data[r][c] = word[i]
                    self.occupied.add((r, c))
                return True
        return False

    def insert_connector_x(self, max_connectors=2):
        """
        Inserta hasta max_connectors letras 'X' cerca de palabras clave,
        para permitir conexiones en ángulo de 90º.
        """
        candidates = []
        for r in range(1, self.rows - 1):
            for c in range(1, self.columns - 1):
                if (r, c) in self.occupied:
                    continue
                # Busca celdas rodeadas de letras clave
                surroundings = [
                    self.data[r][c - 1], self.data[r][c + 1],
                    self.data[r - 1][c], self.data[r + 1][c]
                ]
                keyword_count = sum(1 for l in surroundings if l in "LOKTA")
                if keyword_count >= 2:
                    candidates.append((r, c))

        random.shuffle(candidates)
        inserted = 0
        for (r, c) in candidates:
            if inserted >= max_connectors:
                break
            self.data[r][c] = 'X'
            self.occupied.add((r, c))
            inserted += 1

    def is_keyword_near(self, r, c, direction):
        """
        Verifica si hay una palabra clave adyacente en la dirección dada desde (r, c).
        """
        offsets = {
            'right': [(0, i) for i in range(1, 5)],
            'left': [(0, -i) for i in range(1, 5)],
            'down': [(i, 0) for i in range(1, 5)],
            'up': [(-i, 0) for i in range(1, 5)]
        }
        for dr, dc in offsets[direction]:
            rr, cc = r + dr, c + dc
            if 0 <= rr < self.rows and 0 <= cc < self.columns:
                sequence = self.get_sequence_from(rr, cc, direction)
                if sequence in self.keywords or sequence[::-1] in self.keywords:
                    return True
        return False

    def get_sequence_from(self, r, c, direction):
        """
        Extrae hasta 4 letras desde una posición y dirección.
        Devuelve la secuencia de letras.
        """
        sequence = ''
        for i in range(4):  # Máximo largo de keyword
            if 0 <= r < self.rows and 0 <= c < self.columns:
                sequence += self.data[r][c]
                if direction == 'right':
                    c += 1
                elif direction == 'left':
                    c -= 1
                elif direction == 'down':
                    r += 1
                elif direction == 'up':
                    r -= 1
            else:
                break
        return sequence

    def fill_with_letters(self):
        """
        Rellena las celdas vacías con letras aleatorias.
        """
        for r in range(self.rows):
            for c in range(self.columns):
                if self.data[r][c] == '0':
                    self.data[r][c] = random.choice(self._valid_letters)

    def display(self):
        """
        Imprime el tablero en consola.
        """
        for row in self.data:
            print(' '.join(row))

#### Prueba en Ejecución de la tabla

In [3]:
# Ejemplo de sencillo de uso
table = Table(5, 5)
table.display()

X T Z I Z
O A C U T
I J K L L
R W X O O
L O K K K


### Objeto Game

---

**El Objeto `Game`** debe de tener las siguientes características:

* Se debe poder modificar la tabla, mediante las palabras clave, osease, que se tachen las letras, hacer una lista de letras tachadas (solo las que conforman las palabras clave [`LOK`,`TLAK`,`TAK`], para qiue se de a entender que estan tachadas, se utilizaran corchetes []), las letras que serán elegidas para oscurecer, se reemplazará por un cuadrado negro (`⬛`).

* Que detecte si el usuario intenta oscurecer varias veces un mismo recuadro, y que diga que el movimiento es inválido, así miso al tratar de tachar una letra ya tachada.

* Si el usuario intenta tachar una palabra o letra incorrecta, que lo detecte.

* Que detecte cuando el usuario intente utilizar el `X` como conector, y que solo lo permita utilizar en 90º

* Que detecte cuando todas las celdas estén tanto tachadas (en el caso de las `keywords`), y oscurecidas en caso de las demás letras. Esto podria ser:

    - Que se guarde contabilicen las keywords y las letras.

    - Luego que mediante cada keywords que se tachen y letras oscurecidas se guarden en listas individuales, llamadas `keywords_complete` y `black_letters`.

    - Cuando estas esten completas, debido a que se contabilizaron anteriormente, el juego cerrara, debido a que ya se cumplio el objetivo del juego

* Que el jugador tenga la opción de rendirse en cualquier momento que desee.

* Que haya un menu de opciones en donde, pueda seleccionarse si se tachara (en caso de que seleccione esa opcion, el jugador debera seleccionar las celdas que tachará, luego de eso se le preguntará cual o cuales celdas oscurecerá), y tambien tendra la opción de rendirse, a lo cual se le dirá si está seguro o no.

---

In [4]:
class Game:
    def __init__(self, rows, columns):
        self.table = Table(rows, columns)
        self.keyword_cells = []
        self.black_cells = []  # Celdas oscurecidas
        self.running = True

    def clear_screen(self):
        try:
            os.system('cls' if os.name == 'nt' else 'clear')
        except:
            print("\n" * 50)

    def display_board(self):
        print("\n📦 TABLERO ACTUAL:\n")
        for r in range(self.table.rows):
            row_display = []
            for c in range(self.table.columns):
                coord = (r, c)
                char = self.table.data[r][c]
                if coord in self.keyword_cells:
                    row_display.append(f"[{char}]")
                elif coord in self.black_cells:
                    row_display.append("⬛ ")  # Celda oscurecida
                else:
                    row_display.append(f" {char} ")
            print(''.join(row_display))

    def ask_coordinates(self, length):
        while True:
            entry = input(f"Ingrese las {length} coordenadas (formato fila,columna separadas por espacio):\n> ").strip()
            parts = entry.split()
            if len(parts) != length:
                print(f"❌ Debes ingresar exactamente {length} coordenadas.")
                continue
            try:
                coords = [tuple(map(int, p.split(','))) for p in parts]
                if all(0 <= r < self.table.rows and 0 <= c < self.table.columns for r, c in coords):
                    return coords
                else:
                    print("❌ Alguna coordenada está fuera del tablero.")
            except Exception:
                print("❌ Formato inválido. Ejemplo válido: 0,1 0,2 0,3")

    def is_straight_line(self, coords):
        rows = [r for r, _ in coords]
        cols = [c for _, c in coords]
        return len(set(rows)) == 1 or len(set(cols)) == 1

    def is_consecutive(self, coords):
        coords = sorted(coords)
        if self.is_straight_line(coords):
            if len(set(r for r, _ in coords)) == 1:  # Horizontal
                cols = sorted([c for _, c in coords])
                return all(cols[i] + 1 == cols[i+1] for i in range(len(cols)-1))
            elif len(set(c for _, c in coords)) == 1:  # Vertical
                rows = sorted([r for r, _ in coords])
                return all(rows[i] + 1 == rows[i+1] for i in range(len(rows)-1))
        return False

    def extract_word(self, coords):
        return ''.join([self.table.data[r][c] for r, c in coords])

    def validate_connector_x(self, coords):
        x_coords = [coord for coord in coords if self.table.data[coord[0]][coord[1]] == 'X']
        if len(x_coords) != 1:
            return False

        x_r, x_c = x_coords[0]
        horiz = sorted([coord for coord in coords if coord[0] == x_r])
        vert = sorted([coord for coord in coords if coord[1] == x_c])

        if len(horiz) < 2 or len(vert) < 2:
            return False

        horiz_word = ''.join(self.table.data[r][c] for r, c in horiz)
        vert_word = ''.join(self.table.data[r][c] for r, c in vert)

        valid_keywords = ['LOK', 'TLAK', 'TAK', 'TA']
        return (
            any(horiz_word == k or horiz_word[::-1] == k for k in valid_keywords) or
            any(vert_word == k or vert_word[::-1] == k for k in valid_keywords)
        )

    def mark_keyword(self):
        length = input("¿Cuántas letras tiene la palabra que quieres tachar? (incluye la X si la usas como conector): ").strip()
        if length not in ['2', '3', '4', '5']:
            print("❌ Longitud inválida.")
            input("Presiona ENTER para continuar...")
            return

        length = int(length)
        coords = self.ask_coordinates(length)

        if any(coord in self.keyword_cells for coord in coords):
            print("❌ Ya tachaste alguna de esas letras.")
            input("Presiona ENTER para continuar...")
            return

        letters = [self.table.data[r][c] for r, c in coords]

        if 'X' in letters:
            if not self.validate_connector_x(coords):
                print("❌ No puedes tachar la letra 'X'. Solo se puede usar como conector válido en ángulo de 90° y formando una palabra clave.")
                input("Presiona ENTER para continuar...")
                return
            else:
                coords_to_mark = [coord for coord in coords if self.table.data[coord[0]][coord[1]] != 'X']
                self.keyword_cells.extend(coords_to_mark)
                print("✅ Se utilizó la X correctamente como conector. Letras tachadas.")
                word = self.extract_word(coords_to_mark)
                self.apply_word_effect(word, coords_to_mark)
                return

        if not self.is_straight_line(coords):
            print("❌ Las coordenadas no están en línea recta.")
            input("Presiona ENTER para continuar...")
            return
        if not self.is_consecutive(coords):
            print("❌ Las coordenadas deben estar juntas (celdas contiguas).")
            input("Presiona ENTER para continuar...")
            return

        word = ''.join(letters)
        word_rev = word[::-1]
        valid_keywords = ['LOK', 'TLAK', 'TAK', 'TA']
        if word not in valid_keywords and word_rev not in valid_keywords:
            print(f"❌ La palabra '{word}' no es válida.")
            input("Presiona ENTER para continuar...")
            return

        self.keyword_cells.extend(coords)
        print(f"✅ Palabra '{word}' tachada correctamente.")
        self.apply_word_effect(word if word in valid_keywords else word_rev, coords)

    def apply_word_effect(self, word, coords_marked):
        self.clear_screen()
        self.display_board()
        word = word.upper()
        if word == 'LOK':
            print("Debes oscurecer 1 celda extra (que no sea parte de una palabra clave tachada ni ya oscurecida).")
            self.apply_extra_black(1, coords_marked)
        elif word == 'TLAK':
            print("Debes oscurecer 2 celdas extra adyacentes (que no sean parte de una palabra clave tachada ni ya oscurecida ni de TAK).")
            self.apply_extra_black(2, coords_marked, adjacent=True)
        elif word == 'TAK':
            print("Debes oscurecer todas las celdas con la misma letra (elige la letra a oscurecer).")
            self.apply_black_all_letter()
        elif word == 'TA':
            print("Debes oscurecer todas las celdas con la misma letra (elige la letra a oscurecer).")
            self.apply_black_all_letter()

        # Mostrar la tabla actualizada antes de pedir ENTER
        self.display_board()
        if self.player_wins():
            print("🎉 ¡Felicidades! Has oscurecido todas las celdas y tachado todas las palabras clave. ¡Ganaste!")
            self.running = False
            input("Presiona ENTER para salir...")
        else:
            input("Presiona ENTER para volver al menú...")

    def apply_extra_black(self, amount, coords_marked, adjacent=False):
        extras = []
        while len(extras) < amount:
            self.display_board()
            entry = input(f"Ingrese la coordenada extra a oscurecer ({len(extras)+1}/{amount}) (formato fila,columna):\n> ").strip()
            try:
                r, c = map(int, entry.split(','))
                coord = (r, c)
                if coord in self.keyword_cells or coord in self.black_cells or coord in coords_marked or coord in extras:
                    print("❌ Celda inválida o ya oscurecida/tachada.")
                    continue
                if adjacent and len(extras) == 1:
                    r0, c0 = extras[0]
                    if not ((abs(r0 - r) == 1 and c0 == c) or (abs(c0 - c) == 1 and r0 == r)):
                        print("❌ Las celdas deben ser adyacentes.")
                        continue
                extras.append(coord)
            except Exception:
                print("❌ Formato inválido. Ejemplo válido: 1,2")
        self.black_cells.extend(extras)

    def apply_black_all_letter(self):
        letter = input("¿Qué letra deseas oscurecer en todo el tablero? (escribe la letra): ").strip().upper()
        if len(letter) != 1 or not letter.isalpha():
            print("❌ Letra inválida.")
            return
        new_blacks = []
        for r in range(self.table.rows):
            for c in range(self.table.columns):
                coord = (r, c)
                if self.table.data[r][c] == letter and coord not in self.keyword_cells and coord not in self.black_cells:
                    new_blacks.append(coord)
        if not new_blacks:
            print("❌ No hay celdas con esa letra para oscurecer.")
        else:
            self.black_cells.extend(new_blacks)
            print(f"✅ Se oscurecieron todas las '{letter}' del tablero.")

    def player_wins(self):
        total = self.table.rows * self.table.columns
        return len(set(self.keyword_cells + self.black_cells)) == total

    def menu(self):
        while self.running:
            self.display_board()
            print("\n📋 MENÚ PRINCIPAL:")
            menu_options = [
                ('Tachar una palabra clave', 'mark'),
                ('Rendirse', 'surrender')
            ]
            for idx, (text, _) in enumerate(menu_options, 1):
                print(f"{idx}. {text}")
            choice = input(f"\nSelecciona una opción (1-{len(menu_options)}): ").strip()
            if not choice.isdigit():
                print("❌ Debes ingresar un número.")
                input("Presiona ENTER para continuar...")
                continue
            idx = int(choice) - 1
            if 0 <= idx < len(menu_options):
                _, key = menu_options[idx]
                # self.clear_screen()  # <--- ELIMINADA
                if key == 'mark':
                    self.mark_keyword()
                elif key == 'surrender':
                    confirm = input("¿Seguro que quieres rendirte? (s/n): ").strip().lower()
                    if confirm == 's':
                        print("💀 Te has rendido. Fin del juego.")
                        self.running = False
                    else:
                        print("👍 Continuamos...")
                        input("Presiona ENTER para continuar...")
            else:
                print("❌ Opción no válida. Intenta de nuevo.")
                input("Presiona ENTER para continuar...")

    def start_game(self):
        print("🕹️ Iniciando el juego...\n")
        self.table.insert_keyword("LOK", direction='horizontal')
        self.table.insert_keyword("TLAK", direction='vertical', reversed=True)
        self.table.insert_keyword("TA", direction=random.choice(['horizontal', 'vertical']))
        self.table.fill_with_letters()
        self.menu()

In [6]:
# Iniciar el juego
juego = Game(6, 6)
juego.start_game()

🕹️ Iniciando el juego...


📦 TABLERO ACTUAL:

 C  E  X  P  Q  T 
 T  A  K  O  L  A 
 L  X  K  A  L  T 
 O  X  K  O  L  K 
 K  G  L  R  V  O 
 T  L  O  K  A  L 

📋 MENÚ PRINCIPAL:
1. Tachar una palabra clave
2. Rendirse
❌ Longitud inválida.

📦 TABLERO ACTUAL:

 C  E  X  P  Q  T 
 T  A  K  O  L  A 
 L  X  K  A  L  T 
 O  X  K  O  L  K 
 K  G  L  R  V  O 
 T  L  O  K  A  L 

📋 MENÚ PRINCIPAL:
1. Tachar una palabra clave
2. Rendirse
❌ Longitud inválida.

📦 TABLERO ACTUAL:

 C  E  X  P  Q  T 
 T  A  K  O  L  A 
 L  X  K  A  L  T 
 O  X  K  O  L  K 
 K  G  L  R  V  O 
 T  L  O  K  A  L 

📋 MENÚ PRINCIPAL:
1. Tachar una palabra clave
2. Rendirse
❌ Longitud inválida.

📦 TABLERO ACTUAL:

 C  E  X  P  Q  T 
 T  A  K  O  L  A 
 L  X  K  A  L  T 
 O  X  K  O  L  K 
 K  G  L  R  V  O 
 T  L  O  K  A  L 

📋 MENÚ PRINCIPAL:
1. Tachar una palabra clave
2. Rendirse
❌ Longitud inválida.

📦 TABLERO ACTUAL:

 C  E  X  P  Q  T 
 T  A  K  O  L  A 
 L  X  K  A  L  T 
 O  X  K  O  L  K 
 K  G  L  R  V  O 
 T 